<h1><center>Function check_areas.py</center>
<h3>Parameter used in check_areas:</h3>
<ul>
	<li>path:</li>
</ul>

<h3>Function used in check_areas:</h3>
<ul>
    <li>le_interp(time[kk],area_c[kk,j])</li>
</ul>  

In [1]:
from library import *

Using TensorFlow backend.


In [ ]:
time = np.loadtxt(path+'time.csv')
area_c = np.loadtxt(path+'area_c.csv')
alpha_mu_spot = np.loadtxt(path+'alpha_mu_spot.csv')
#alpha_mu_spot = np.loadtxt(path+'FileName.txt')

alpha_mu_spot = np.reshape(alpha_mu_spot,(np.size(time),6,11))

#area_c = area_c.reshape([np.size(time),6]) # UMA OPÇÃO AO IF QUE VEM EM SEGUIDA, NECESSÁRIO PARA SOMAR OS VALORES NO CASO DA AREA_C SER UMA MATRIZ UNIDIMENSIONAL

if area_c.ndim > 1:
    count = np.nansum(area_c,1)
else:
    count = np.nansum(area_c)

mu = np.nanmean(count)
sigma = np.nanstd(count)

n=count.size

meanMat = matlib.repmat(mu, n, 1)
sigmaMat = matlib.repmat(sigma, n, 1)

count = count.reshape(meanMat.shape)

outliers = np.abs(count - meanMat) > (3*sigmaMat)

area_c[np.any(outliers,1),:] = np.nan

alpha_mu_spot[np.any(outliers,1),:] = np.nan


'''QUAL É O OBJETIVO DAS PRÓXIMAS LINHAS (DE 129 À 143) DE CÓDIGO? ESTÁ DANDO ERRO PARA kk == 0 ''' 
ck, kk = np.unique(time, return_index=True)

'''VERIFICAR AQUI SE OS VALORES RECEBIDOS POR KK SÃO REALMENTE OS PREVISTOS'''

n = area_c.shape[1]

for j in range(n):
    area_c[kk,j] = le_interp(time[kk],area_c[kk,j])


n = alpha_mu_spot.shape

for j in range(n[1]):
    for i in range(n[2]):
        alpha_mu_spot[kk,j,i] = le_interp(time[kk],alpha_mu_spot[kk,j,i])

print('\nArquivos salvos em '+path+' :\n')
np.savetxt(path+'\\check_areas_time.csv', time)
print('check_areas_time.csv\n')

np.savetxt(path+'\\check_areas_area_c.csv', area_c)
print('check_areas_area_c.csv\n')

with open(path+'\\check_areas_alpha_mu_spot.csv', 'w') as outfile:
    # I'm writing a header here just for the sake of readability
    # Any line starting with "#" will be ignored by numpy.loadtxt
    outfile.write('# Array shape: {0}\n'.format(alpha_mu_spot.shape))

    # Iterating through a ndimensional array produces slices along
    # the last axis. This is equivalent to data[i,:,:] in this case
    for dataSlice in alpha_mu_spot:

        # Writing out a break to indicate different slices...
        outfile.write('# New slice\n')

        np.savetxt(outfile, dataSlice, fmt='%-7.5f')


print('check_areas_alpha_mu_spot.csv\n')

return time, area_c, alpha_mu_spot
